In [3]:
# Thêm thư viện
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model , Sequential
# from keras.layers.core import Dense
# from keras.layers.core import Dropout
# from keras.layers.core import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import string

#Warnings
import warnings
warnings.filterwarnings('ignore')

Rename file 
- Từ tập dữ liệu dataset , gom rename các file trong folder train lại sau đó cho tiếp các file test vào folder train để name lại một lần nữa 

In [6]:
def rename_files(folder_path):
    # Lặp qua các thư mục con trong folder_path
    for label_folder in os.listdir(folder_path):
        label_folder_path = os.path.join(folder_path, label_folder)
        if os.path.isdir(label_folder_path):
            # Lặp qua các tệp trong thư mục con
            for filename in os.listdir(label_folder_path):
                file_path = os.path.join(label_folder_path, filename)
                if os.path.isfile(file_path):
                    # Tạo số ngẫu nhiên từ 0 đến 999999
                    random_number = random.randint(0, 999999)
                    # Tạo tên mới
                    new_filename = label_folder + "_" + str(random_number) + os.path.splitext(filename)[1]
                    new_file_path = os.path.join(label_folder_path, new_filename)
                    # Đổi tên tệp
                    os.rename(file_path, new_file_path)

# Gọi hàm và truyền đường dẫn đến thư mục chứa các thư mục nhãn
rename_files('./Dataset_fine_tuning/train')


Chia train , test = 7:3
- Tôi có folder './Dataset_fine_tuning/ trong folder này là các folder với tên là nhãn tương ứng  và trong các folder đó là các file , giờ hãy tạo trong folder Dataset_fine_tuning 2 folder là train và test , sau đó chia các dữ liệu ban đầu thành 7:3 vào 2 folder đó cho tôi 

In [8]:
# Đường dẫn tới thư mục chứa dữ liệu ban đầu
data_folder = './Dataset_fine_tuning/'

# Tạo thư mục train và test nếu chưa tồn tại
train_folder = os.path.join(data_folder, 'train')
test_folder = os.path.join(data_folder, 'test')
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Lấy danh sách các thư mục nhãn
label_folders = [folder for folder in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, folder))]

# Duyệt qua từng thư mục nhãn
for label in label_folders:
    label_path = os.path.join(data_folder, label)
    
    # Lấy danh sách các file trong thư mục nhãn
    files = [f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))]
    
    # Shuffle danh sách các file
    random.shuffle(files)
    
    # Tính chỉ số để chia tỷ lệ 7:3
    split_index = int(0.7 * len(files))
    
    # Phân chia danh sách thành tập train và test
    train_files = files[:split_index]
    test_files = files[split_index:]
    
    # Tạo thư mục nhãn tương ứng trong thư mục train và test
    train_label_folder = os.path.join(train_folder, label)
    test_label_folder = os.path.join(test_folder, label)
    os.makedirs(train_label_folder, exist_ok=True)
    os.makedirs(test_label_folder, exist_ok=True)
    
    # Di chuyển các file vào thư mục train và test
    for file in train_files:
        shutil.move(os.path.join(label_path, file), os.path.join(train_label_folder, file))
    for file in test_files:
        shutil.move(os.path.join(label_path, file), os.path.join(test_label_folder, file))

print("Chia dữ liệu thành công!")


Chia dữ liệu thành công!


Chia train : valid : test = 7:2:1

In [10]:
import os
import shutil
import random

# Đường dẫn tới thư mục chứa dữ liệu ban đầu
data_folder = './Dataset_fine_tuning_train_valid_test/'

# Tạo thư mục train, validation và test nếu chưa tồn tại
train_folder = os.path.join(data_folder, 'train')
valid_folder = os.path.join(data_folder, 'valid')
test_folder = os.path.join(data_folder, 'test')
os.makedirs(train_folder, exist_ok=True)
os.makedirs(valid_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Lấy danh sách các thư mục nhãn
label_folders = [folder for folder in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, folder))]

# Duyệt qua từng thư mục nhãn
for label in label_folders:
    label_path = os.path.join(data_folder, label)
    
    # Lấy danh sách các file trong thư mục nhãn
    files = [f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))]
    
    # Shuffle danh sách các file
    random.shuffle(files)
    
    # Tính chỉ số để chia tỷ lệ 7:2:1
    train_index = int(0.7 * len(files))
    valid_index = int(0.9 * len(files))
    
    # Phân chia danh sách thành tập train, validation và test
    train_files = files[:train_index]
    valid_files = files[train_index:valid_index]
    test_files = files[valid_index:]
    
    # Tạo thư mục nhãn tương ứng trong thư mục train, validation và test
    train_label_folder = os.path.join(train_folder, label)
    valid_label_folder = os.path.join(valid_folder, label)
    test_label_folder = os.path.join(test_folder, label)
    os.makedirs(train_label_folder, exist_ok=True)
    os.makedirs(valid_label_folder, exist_ok=True)
    os.makedirs(test_label_folder, exist_ok=True)
    
    # Di chuyển các file vào thư mục train, validation và test
    for file in train_files:
        shutil.move(os.path.join(label_path, file), os.path.join(train_label_folder, file))
    for file in valid_files:
        shutil.move(os.path.join(label_path, file), os.path.join(valid_label_folder, file))
    for file in test_files:
        shutil.move(os.path.join(label_path, file), os.path.join(test_label_folder, file))

print("Chia dữ liệu thành công!")


Chia dữ liệu thành công!
